In [20]:
'''%pip install --disable-pip-version-check \
        torch==1.13.1 \
        torchdata==0.5.1 --quiet

%pip install \
        transformers==4.27.2 \
        datasets==2.11.0 --quiet'''

'%pip install --disable-pip-version-check         torch==1.13.1         torchdata==0.5.1 --quiet\n\n%pip install         transformers==4.27.2         datasets==2.11.0 --quiet'

In [21]:
'''%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet'''

'%pip install --upgrade pip\n%pip install --disable-pip-version-check     torch==1.13.1     torchdata==0.5.1 --quiet\n\n%pip install     transformers==4.27.2     datasets==2.11.0  --quiet'

In [3]:
from config import reddit_client_id, reddit_client_secret, reddit_user_agent

In [4]:
import praw
# Initialize the Reddit API wrapper
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_client_secret,
                     user_agent=reddit_user_agent)

# Replace 'python' with the name of the subreddit you want to pull posts from
subreddit = reddit.subreddit('CryptoCurrency')

for submission in reddit.subreddit("CryptoCurrency").hot(limit=10):
    print(submission.title)

#TODO: extract headings and top level comments for top n threads (hot n?)
#TODO: store this where? Patch as json to some store - blob?

AMA Announcement: Unveiling the Future of Decentralized AI and Software Development with Tau Net ($AGRS) - March 25th, 6pm UTC - 1500 USDT Giveaway
Tutorial: Bridging MOON to Arbitrum One via Celer (Celer Bridge is now live!)
Ethereum Could Top $14,000 Next Year Alongside Bitcoin Boom: Standard Chartered
I analyzed the Last 3 Bitcoin halvings here's what I think will happen after the 2024 halving
Cardano Gets First Fiat-Backed Stablecoin with USDM
Daily Crypto Discussion - March 25, 2024 (GMT+0)
This Week in Crypto Games: 'Crypto Valleys' Blasts Off, Immutable and Polygon Plot $100M Fund
Litecoin is breaking out of 11Y old LTC/BTC ratio downtrend, next 1W candle on 25th March 2024
Professional Shitposting
Skybridge Capital Founder Advises 'Act Like You're Dead With Your Bitcoin and Don't Sell' 


In [ ]:
'''%pip install emoji'''

In [213]:
# compile the dataset
# convert emojis to plain text before writing to csv
from emoji import demojize
import re


submissions_dataset = []
comments_dataset = []

for submission in reddit.subreddit("CryptoCurrency").top('day',limit=10):
    submissions_dataset.append([submission.id, submission.title, submission.created, submission.score])
    for top_level_comment in submission.comments:
            comments_dataset.append([top_level_comment.id, re.sub(r'\s+', ' ', demojize(top_level_comment.body)), top_level_comment.created_utc, top_level_comment.score, submission.id])

C:\Users\olgakh\AppData\Local\Temp\ipykernel_14216\1999565252.py:10: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  for submission in reddit.subreddit("CryptoCurrency").top('day',limit=10):


In [214]:
import csv

submissions_filename = "reddit_submissions_data.csv"
comments_filename = "reddit_comments_data.csv"

# Define the fields and rows
submission_fields = ['ID', 'Title', 'Unixtime', 'Upvotes']
comment_fields = ['ID','Comment', 'Unixtime', 'Upvotes', 'SubmissionID']


with open(submissions_filename, 'a') as csvfile:
    # Create a writer object
    csvwriter = csv.writer(csvfile)
    # Write the fields and rows to the file
    csvwriter.writerow(submission_fields)
    csvwriter.writerows(submissions_dataset)
    
with open(comments_filename, 'a', encoding="utf-8") as csvfile:
    # Create a writer object
    csvwriter = csv.writer(csvfile)
    # Write the fields and rows to the file
    csvwriter.writerow(comment_fields)
    csvwriter.writerows(comments_dataset)


In [215]:
import pandas as pd
pd.read_csv(submissions_filename)

,ID,Title,Unixtime,Upvotes
0,1bmshtm,"Ethereum Could Top $14,000 Next Year Alongside...",1.711307e+09,567
1,1bmkto3,Cardano Gets First Fiat-Backed Stablecoin with...,1.711287e+09,442
2,1bn4dun,I analyzed the Last 3 Bitcoin halvings here's ...,1.711339e+09,220
3,1bmih6w,Litecoin is breaking out of 11Y old LTC/BTC ra...,1.711279e+09,170
4,1bmiena,How to take profits so we can maximize our cry...,1.711279e+09,110
5,1bmj383,EU Anti-Money Laundering Laws Ban Provision of...,1.711282e+09,88
6,1bmqcyu,This Week in Crypto Games: 'Crypto Valleys' Bl...,1.711302e+09,80
7,1bmgrey,What consumer applications do you think will a...,1.711272e+09,47
8,1bmo4nn,The memecoin portfolio experiment. 3rd week,1.711296e+09,40
9,1bmi3xr,Binance charges us more than you think on FIAT...,1.711278e+09,34


In [216]:
pd.read_csv(comments_filename)

,ID,Comment,Unixtime,Upvotes,SubmissionID
0,kwdnu7z,Ethereum [pros](/r/CryptoCurrency/comments/1bm...,1.711308e+09,1,1bmshtm
1,kwdqtko,Nobody knows shit,1.711309e+09,331,1bmshtm
2,kwdpxif,Ethereum could also top 20k. Or go back to 700...,1.711308e+09,549,1bmshtm
3,kwdsan3,Wtf is the point of following this sub every p...,1.711309e+09,137,1bmshtm
4,kwdqa7o,Could.,1.711309e+09,15,1bmshtm
...,...,...,...,...,...
411,kwfv14l,Joining the crypto gambleverse to dreaming to ...,1.711339e+09,1,1bmi3xr
412,kwg5z9w,"yes it is spread, welcome to the world of hidd...",1.711345e+09,1,1bmi3xr
413,kwbqla5,Ran away from Binance the moment they stopped ...,1.711280e+09,0,1bmi3xr
414,kwboc7t,I had a balance of 980€ in BTC after buying 10...,1.711278e+09,-6,1bmi3xr


Note: # From qualitative analysis: a comment doesn't often have multiple coins+strong sentiment associated with each one, usually it's one coin mention and discussion, it may mention other coins but the comment itself will center around one of them and only have strong sentiment associated with that one coin. Comments like "BTC is amazing, Etherium sucks, Doge is the best" are rare. 

In [ ]:
#TODO: what are we extracting? Coin mentions, sentiment??


In [ ]:
# TODO: BTC/Bitcoin mentions - need to map name/abbreviation to the same coin
# Can potentially be done by the LLM
# If not, can scrape the coinwire page and write a function to map

In [49]:
#TODO: load the base model using tensorflow

# Loading the base model, the tokenizer and the config
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [50]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [51]:
# Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, batched = True)

In [52]:
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-base', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>'

In [ ]:
#TODO: write a prompt

prompt = f'''For a given Reddit comment, extract the Cryptocurrency the comment mentions and the sentiment associated with that cryptocurrency. 
If there are multiple Cryptocurrencies mentioned in the comment, extract the main Cryptocurrency that the commenter is focusing on.
Classify the sentiment associated with this Cryptocurrency into positive, negative or neutral.

Comment: {}'''